# Import libs

In [ ]:
from functools import reduce
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Create a dataframe list with sheets values

In [ ]:
months_list = [
    "JANEIRO",
    "FEVEREIRO",
    "MARÇO",
    "ABRIL",
    "MAIO",
    "JUNHO",
    "JULHO",
    "AGOSTO",
    "SETEMBRO",
    "OUTUBRO",
    "NOVEMBRO",
    "DEZEMBRO"
]

# Lista para armazenar os dados de todas as planilhas
# data = list()
# Loop para percorrer os arquivos de 2016 a 2023
for year in range(2016, 2023):
    arquivo_excel = f"sheets/CONTROLE CHEQUE {year}.xlsx"  # Nome do arquivo Excel para cada ano
    arquivo_excel_destino = f"sheets/CONTROLE CHEQUE {year}_mask.xlsx"
    # Loop para percorrer as abas de cada arquivo
    with pd.ExcelWriter(arquivo_excel_destino) as writer:  # doctest: +SKIP
        for month in months_list:  
            try:
                # Ler a planilha atual e adicionar os dados à lista
                df = pd.read_excel(arquivo_excel, sheet_name=month)
                columns_to_clean =['CLIENTE', 'OBSERVAÇÃO', 'DESTINO']
                df['Nº CHEQUE'] = df['Nº CHEQUE'].astype('category').cat.codes + 1

                for coluna in columns_to_clean:
                    df[coluna] = df[coluna].str[0] + df[coluna].str[-1]
                
                df.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1).to_excel(writer, sheet_name=month, index=False)

                # df.to_excel(arquivo_excel_destino, index=False, sheet_name=month)
            except:
                print(f"Mês {month} de {year} vazio ou não existe na planilha")
# 2023 Unnamed: 11 a 15, outros anos Unnamed: 2 e 3

In [ ]:
arquivo_excel = f"sheets/CONTROLE CHEQUE 2023.xlsx"  # Nome do arquivo Excel para cada ano
arquivo_excel_destino = f"sheets/CONTROLE CHEQUE 2023_mask.xlsx"
# Loop para percorrer as abas de cada arquivo
with pd.ExcelWriter(arquivo_excel_destino) as writer:  # doctest: +SKIP
    for month in months_list:  
        try:
            # Ler a planilha atual e adicionar os dados à lista
            df = pd.read_excel(arquivo_excel, sheet_name=month)
            columns_to_clean =['CLIENTE', 'OBSERVAÇÃO', 'DESTINO']
            df['Nº CHEQUE'] = df['Nº CHEQUE'].astype('category').cat.codes + 1

            for coluna in columns_to_clean:
                df[coluna] = df[coluna].str[0] + df[coluna].str[-1]
            
            df.drop(['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], axis=1).to_excel(writer, sheet_name=month, index=False)

            # df.to_excel(arquivo_excel_destino, index=False, sheet_name=month)
        except:
            print(f"Mês {month} de {year} vazio ou não existe na planilha")

# Merge dataframe list

In [ ]:
# Concatenar os dados de todas as planilhas em um único DataFrame
concat_data = pd.concat(data)

concat_data.columns

# Config raw data

In [ ]:
raw_columns_list = [
    "CLIENTE",
    "BANCO",
    "Nº CHEQUE",
    "VALOR (R$)",
    "VENCIMENTO",
    "SAÍDA",
    "COD",
    "DESTINO",
    "OBSERVAÇÃO"
]


REQUIRED_COLUMNS = 5 # CLIENTE, BANCO, Nº CHEQUE, VALOR (R$), VENCIMENTO

raw_data = concat_data[raw_columns_list].dropna(thresh=REQUIRED_COLUMNS)
db_user = 'root'
db_name = 'Checks'
db_password = '123456'
db_host = 'localhost'
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

raw_table_name = 'raw_data'

# Save raw_data in SQL

In [ ]:
# # Use o método to_sql para salvar o DataFrame no banco de dados MySQL
raw_data.to_sql(
    raw_table_name,
    con=engine,
    index=True,
    if_exists='replace'
)

# print(f'Dados salvos na tabela {raw_table_name} no banco de dados MySQL.')

# Get raw_data info

In [ ]:
raw_data.info()

# Drop null values if CLIENTE and Nº Cheque is null
# IMPORTANTE: 
* ## adicionar COD? 
* ## **DECISÃO**: Não, registros novos não tem cod. Um caso de registro antigo sem cod corrigido ao longo do caminho

In [ ]:
# Drop null values if CLIENTE or Nº Cheque is null
accur_data = raw_data.dropna(subset=['CLIENTE', 'BANCO'], how='any')

accur_data.info()

# Drop duplicates values if CLIENTE and Nº Cheque are equal
## IMPORTANTE: 
* # Será que não é bom ter duplicadas nessa situação para testar a tabela fato?
* ## **DECISÃO**: Por enquanto sim


In [ ]:
# dropar duplicatas se cliente, banco e nº cheque forem iguais, mantendo a que tiver valor na coluna saída
accur_data = accur_data.drop_duplicates(subset=['CLIENTE', 'BANCO', 'Nº CHEQUE', 'SAÍDA'], keep='last')
accur_data.info()

# Transform BANCO column

In [ ]:
# accur_data['BANCO'].unique()

# array(['ITAU', 'BB', 'CAIXA', 'BNB', 'SICOOB', 'BRADESCO', 'BD',
#        'SANTANDER', 'itau', 'SICOB', 'UNICRED', ' BD', 'HSBC',
#        'ITAU       ', 'BB ', 'BANESE', 'SICREDI', 'CITIBANK', 'ASCOOB',
#        'SICRED', 'SAFRA', 'SICREDII', 'BS'], dtype=object)

banks_adjust_dict = {
    'BB': 'BANCO DO BRASIL',
    'BNB': 'BANCO DO NORDESTE',
    'SICOB': 'SICOOB',
    'BD': 'BRADESCO',
    'BS': 'BRADESCO',
    'itau': 'ITAU',
    'BD': 'BRADESCO',
    'SICRED': 'SICREDI',
    'SICREDII': 'SICREDI'
}
# Corrigir espaços em branco extras e typos nos nomes dos bancos
accur_data['BANCO'] = accur_data['BANCO'].str.strip()  # Remove espaços em branco no início e no final
accur_data['BANCO'] = accur_data['BANCO'].replace(banks_adjust_dict)  # Substitua os nomes dos bancos com o dicionário

# Agora, você pode verificar os valores únicos novamente
print(accur_data['BANCO'].unique())


# Transform CLIENTE column

# Transform COD column

In [ ]:
# print(accur_data['COD'].unique())

# ['P' 'D' 'T' 'O' 'p' 'P ' nan ' P']
cod_dict = {
    'P': 'PAGAMENTO',
    'D': 'DEPOSITO',
    'O': 'OUTROS',
    'T': 'CAIXA'
}

accur_data['COD'] = accur_data['COD'].str.upper()  # Converta todos os valores para maiúsculas
accur_data['COD'] = accur_data['COD'].str.strip()  # Remova espaços em branco no início e no final
# substituir 'P' por 'PAGO', 'D' por 'DEPOSITO' e 'O' por 'OUTROS'
accur_data['COD'] = accur_data['COD'].replace(cod_dict)
# Substitua os valores nulos por 'caixa'
accur_data['COD'] = accur_data['COD'].fillna('CAIXA')
print(accur_data['COD'].unique())

# Transform DESTINO column

# Transform VALOR (R$) column

In [ ]:
# accur_data['VALOR (R$)'] = accur_data['VALOR (R$)'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
accur_data['VALOR (R$)'] = accur_data['VALOR (R$)'].astype(str).str.replace(',', '.').astype(float)
accur_data.info()

# Transform date columns

In [ ]:
date_columns_list = [
    'VENCIMENTO',
    'SAÍDA'
]

replace_dict = {
    r'\s+': '',
    '00:00:00': '', 
    '29/2/2017': '2017-02-28',
    '29/2/2018': '2018-02-28',
    '29/2/2019': '2019-02-28',
    '31/4/2017': '2017-04-30', 
    '31/4/2018': '2018-04-30', 
    '30/62017': '2017-06-30', 
    '31/9/2017': '2017-09-30', 
    '2017-09-31': '2017-09-30', 
    '31/6/2018': '2018-06-30', 
    '31/9/2018': '2018-09-30',
    ',15/04/2016': '2016-04-15',
    '15/04/2016': '2016-04-15',
    '15/04/201': '2016-04-15',
    '10/06/2016': '2016-06-10',
    # '25/07/216': '2016-07-25',
    # '22/08/216': '2016-08-22',
    # '21/09/216': '2016-09-21',
    # '17/10/216': '2016-10-17',
    '216': '2016',
    '2216': '2016',
    '217': '2017',
    '218': '2018',
    '219': '2019',
    '221': '2021',
    '10/112017': '2017-11-10',
    '227/9/2018': '2018-09-27',
    # '22016-10-07': '2016-10-07',
}

accur_data = reduce(
    lambda df, column: df.assign(**{
        column: df[column].astype(str).replace(replace_dict, regex=True)
    }),
    date_columns_list,
    accur_data
)

accur_data[date_columns_list].head(5)

In [ ]:
accur_data.iloc[87]

In [ ]:
accur_data = reduce(
    lambda df, column: df.assign(**{
        column: pd.to_datetime(df[column], format='mixed')#.dt.strftime('%d/%m/%Y')
    }),
    date_columns_list,
    accur_data
)

accur_data[date_columns_list].head(5)

In [ ]:
accur_data.info()

# Add is_in_cash column

In [ ]:
# Criar coluna booleana is_in_cash sendo false quando o valor da coluna SAÍDA não for nulo, Todos os registros com o mesmo valor para as colunas CLIENTE, BANCO, Nº CHEQUE, VALOR (R$), VENCIMENTO terão o valor FALSE na coluna is_in_cash

common_columns_list = [
    'CLIENTE',
    'BANCO',
    'Nº CHEQUE',
    'VALOR (R$)'
]

checks_out = accur_data[accur_data['SAÍDA'].notnull()][common_columns_list]
# accur_data['is_in_cash'] = np.where(
#     accur_data[['CLIENTE', 'BANCO', 'Nº CHEQUE', 'VALOR (R$)', 'VENCIMENTO']].isin(checks_out).all(axis=1),
#     False,
#     True
# )

# checks_out.head(30)

# accur_data['is_in_cash'] = ~accur_data[common_columns_list].isin(checks_out.to_dict()).all(axis=1)

accur_data['is_in_cash'] = np.where(
    accur_data['CLIENTE'].isin(checks_out['CLIENTE']) &
    accur_data['BANCO'].isin(checks_out['BANCO']) &
    accur_data['Nº CHEQUE'].isin(checks_out['Nº CHEQUE']) &
    accur_data['VALOR (R$)'].isin(checks_out['VALOR (R$)']),
    False,
    True
)

# accur_data['is_in_cash'] = reduce(
#     lambda df, column: df.assign(**{
#         column: np.where(
#             df[column].isin(checks_out[column]),
#             False,
#             True
#         )
#     }),
#     common_columns_list,
#     accur_data
# )

# Substitui valores nulos em 'is_in_cash' por True
# accur_data['is_in_cash'].fillna(True, inplace=True)

# Transform Nº CHEQUE column

In [ ]:
# accur_data['Nº CHEQUE'] = accur_data['Nº CHEQUE'].astype(int)
# accur_data.info()

In [ ]:
raw_data.to_excel('raw_data.xlsx', index=False)
accur_data.to_excel('accur_data.xlsx', index=False)

# Save accur_data in SQL

In [ ]:
# # Use o método to_sql para salvar o DataFrame no banco de dados MySQL
accur_table_name = 'accur_data'
accur_data.to_sql(
    accur_table_name,
    con=engine,
    index=True,
    if_exists='replace'
)

print(f'Dados salvos na tabela {accur_table_name} no banco de dados MySQL.')

# Create primary keys dimensional models columns

In [ ]:
primary_keys_columns = [
    "CLIENTE",
    "BANCO",
    # "DESTINO",
    "COD"
    # "OBSERVAÇÃO"
]

# criar uma coluna para o id do nome considerando o nome do cliente. Nome do cliente pode se repetir
# raw_data['name_id'] = raw_data['CLIENTE'].astype('category').cat.codes + 1

# criar uma coluna para o id das primary_keys_columns. Valores dos campos podem se repetir
model_data = reduce(
    lambda df, col: df.assign(**{f'{col.lower()}_id': df[col].astype('category').cat.codes + 1}),
    primary_keys_columns,
    accur_data
)

model_data.info()
print(model_data.head(5))

# Add name_dim columns

In [ ]:
# CREATE TABLE name_dim(
# 	name_id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
# 	"name" VARCHAR(100) NOT NULL,
# 	is_client BOOL NOT NULL DEFAULT true,
# 	client_name VARCHAR(100) DEFAULT "name"
# );
name_dim_columns = [
    'name_id',
    'name',
    'is_client',
    'client_name'
]

# criar coluna is_client se valor de OBSERVAÇÃO for vazio ou se o valor de CLIENTE estar em OBSERVAÇÃO
model_data['is_client'] = np.where(model_data['OBSERVAÇÃO'].isnull() | model_data['CLIENTE'].isin(model_data['OBSERVAÇÃO']), True, False)

# renomear: client_id para name_id e CLIENTE para name
new_name_dim_columns = {
    'CLIENTE': 'name',
    'cliente_id': 'name_id'
}
model_data = model_data.rename(columns=new_name_dim_columns)

# criar coluna client_name com o valor de name se is_client for true. Se for falso mas o valor de OBSERVAÇÃO estiver na coluna CLIENTE, então o valor de client_name será o valor de OBSERVAÇÃO. Ao contrário deixar vazio
model_data['client_name'] = np.where(
    model_data['is_client'] == True, model_data['name'], 
        np.where(
            model_data['OBSERVAÇÃO'].isin(model_data['name']) & model_data['is_client'] == False, model_data['OBSERVAÇÃO'],
            ''
        )
)

model_data.head(5)

# Add bank_dim columns

In [ ]:
# CREATE TABLE bank_dim(
# 	bank_id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
# 	bank_name VARCHAR(100) NOT NULL
# );
bank_dim_columns = [
    'bank_id',
    'bank_name'
]

new_bank_dim_columns = {
    'BANCO': 'bank_name',
    'banco_id': 'bank_id'
}
model_data = model_data.rename(columns=new_bank_dim_columns)

model_data.head(5)

# Add check_dim columns

In [ ]:
# CREATE TABLE check_dim(
# 	check_id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
# 	check_number INT NOT NULL,
# 	check_amount FLOAT NOT NULL,
# 	maturity_date DATE NOT NULL
# );
check_dim_columns = [
    'check_id',
    'check_number',
    'check_amount',
    'maturity_date'
]

new_check_dim_columns = {
    'Nº CHEQUE': 'check_number',
    'VALOR (R$)': 'check_amount',
    'VENCIMENTO': 'maturity_date'
}

model_data = model_data.rename(columns=new_check_dim_columns)

# criando coluna check_id para cada linha única de check_number, name, bank_name
model_data['check_id'] = (
    model_data.groupby(
        [
            # 'maturity_date',
            'name',
            'check_number',
            'bank_name',
            'check_amount'
        ]
    )
        .ngroup() + 1

)

# model_data['check_id'] = (
#     model_data.maturity_date.sort_values()
#         .str.cat(
#             model_data.name.str.cat(
#                 model_data.check_number.astype(str).str.cat(
#                     model_data.bank_name.str.cat(
#                         model_data.check_amount.astype(str), 
#                         sep='-'
#                     ), 
#                     sep='-'
#                 ), 
#                 sep='-'
#             ), 
#             sep='-'
#         )
#             .astype('category').cat.codes + 1
# )

model_data.head(5)

# Add dest_dim columns

In [ ]:
# CREATE TABLE dest_dim(
# 	destination_id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
# 	destination_name VARCHAR(100) NOT NULL,
# 	destination_type VARCHAR(100) NOT NULL  DEFAULT "CAIXA",
# 	destination_complement VARCHAR(100)
# )
dest_dim_columns = [
    'destination_id',
    # 'destination_name',
    'destination_type',
    # 'destination_complement'
]

new_dest_dim_columns = {
    # 'DESTINO': 'destination_name',
    'cod_id': 'destination_id',
    'COD': 'destination_type',
    # 'OBSERVAÇÃO': 'destination_complement'
}

model_data = model_data.rename(columns=new_dest_dim_columns)
model_data.head(5)

# Transform SAÍDA

In [ ]:
model_data['SAÍDA'] = np.where(
    model_data['SAÍDA'].isnull(),
    model_data['maturity_date'],
    model_data['SAÍDA']
)

model_data = model_data.rename(columns={'SAÍDA': 'registration_date'})
model_data.head(5)

# Transform DESTINO

In [ ]:
model_data['description'] = model_data['DESTINO']

In [ ]:
model_data.drop(columns=['DESTINO', 'OBSERVAÇÃO']).to_excel('model_data.xlsx', index=False)

# Create dim & fact tables
# TO DO:
# **Validar check_id**

In [ ]:
# CREATE TABLE check_fact(
# 	id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
# 	FOREIGN KEY(name_id) REFERENCES name_dim(name_id),
# 	FOREIGN KEY(bank_id) REFERENCES bank_dim(bank_id),
# 	FOREIGN KEY(check_id) REFERENCES check_dim(check_id),
# 	FOREIGN KEY(destination_id) REFERENCES name_dim(dest_dim),
# 	"registration_date" DATE NOT NULL
# )

# criar coluna id para cada linha de model_data
model_data['id'] = model_data.index + 1
check_fact_columns = [
    'id',
    'name_id',
    'bank_id',
    'check_id',
    'destination_id',
    'is_in_cash',
    'description',
    'registration_date'
]

# criar tabelas dimensões e fatos com os valores de model_data
name_dim = model_data[name_dim_columns].drop_duplicates()
bank_dim = model_data[bank_dim_columns].drop_duplicates()
check_dim = model_data[check_dim_columns].drop_duplicates()
dest_dim = model_data[dest_dim_columns].drop_duplicates()
check_fact = model_data[check_fact_columns]#.drop_duplicates()
# continua..
name_dim.to_sql(
    'name_dim',
    con=engine,
    index=False,
    if_exists='replace'
)

bank_dim.to_sql(
    'bank_dim',
    con=engine,
    index=False,
    if_exists='replace'
)

check_dim.to_sql(
    'check_dim',
    con=engine,
    index=False,
    if_exists='replace'
)

dest_dim.to_sql(
    'dest_dim',
    con=engine,
    index=False,
    if_exists='replace'
)

check_fact.to_sql(
    'check_fact',
    con=engine,
    index=False,
    if_exists='replace'
)